<a href="https://colab.research.google.com/github/shivamdhumal77/LLM_Projects/blob/main/rag_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
pip install langchain langchain_openai pandas langchain_community faiss-cpu

In [73]:
pip install rank_bm25

In [75]:
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
import re
from datetime import datetime

# 1. Load and preprocess CSV data
def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path)

    # Convert and format date
    df['Date'] = pd.to_datetime(df['Date'], utc=True).dt.tz_convert(None)
    df['Date'] = df['Date'].dt.strftime('%d-%m-%Y')  # Format: day-month-year

    # Create text data with proper formatting
    text_data = []
    for _, row in df.iterrows():
        entry = f"Date: {row['Date']}"

        # Conditionally add other columns if they exist
        if 'Open' in row and pd.notna(row['Open']):
            entry += f", Open: {row['Open']}"
        if 'High' in row and pd.notna(row['High']):
            entry += f", High: {row['High']}"
        if 'Low' in row and pd.notna(row['Low']):
            entry += f", Low: {row['Low']}"
        if 'Close' in row and pd.notna(row['Close']):
            entry += f", Close: {row['Close']}"
        if 'Volume' in row and pd.notna(row['Volume']):
            entry += f", Volume: {row['Volume']}"

        text_data.append(entry)

    # Combine all text into a single document for proper chunking
    full_text = "\n".join(text_data)
    return df, full_text

# 2. Process and split documents
def process_documents(full_text):
    text_splitter = CharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100,
        separator="\n"
    )
    return text_splitter.split_text(full_text)

# 3. Create vector store
def create_vector_store(docs):
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"  # Fixed model name
    )
    return FAISS.from_texts(docs, embeddings)

# 4. Set up QA chain
def setup_qa_chain(vector_store):
    # Initialize ChatOpenAI model
    llm_1 = ChatOpenAI(
        api_key="ollama",
        base_url="https://sunny-gerri-finsocialdigitalsystem-d9b385fa.koyeb.app/v1",
        model="athene-v2"
    )

    # Custom prompt template
    prompt_template = """Use the following financial data to answer the question:
    {context}

    Question: {question}
    Answer:"""

    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    return RetrievalQA.from_chain_type(
        llm=llm_1,
        chain_type="stuff",
        retriever=vector_store.as_retriever(),
        chain_type_kwargs={"prompt": PROMPT}
    )

# Normalize query to match date format (dd-mm-yyyy)
def normalize_date_query(query):
    # Search for date pattern in query (e.g., 2 Feb 2023 or 02-02-2023)
    date_pattern = r"(\d{1,2})\s*(\w{3})\s*(\d{4})"  # e.g. "2 Feb 2023"
    match = re.search(date_pattern, query.lower())
    if match:
        day = match.group(1).zfill(2)  # Ensure 2 digits
        month_str = match.group(2).title()
        year = match.group(3)
        # Convert month name to number
        month_number = datetime.strptime(month_str, "%b").month
        formatted_date = f"{day}-{month_number:02d}-{year}"
        return formatted_date
    return query  # Return as is if no date pattern found

# Main execution flow
if __name__ == "__main__":
    # Load and preprocess data
    df, data_text = load_and_preprocess_data("/content/AAPL_historical_data.csv")  # Replace with your CSV path

    # Process documents
    documents = process_documents(data_text)

    # Create vector store
    vector_store = create_vector_store(documents)

    # Setup QA chain
    qa_chain = setup_qa_chain(vector_store)

    # Query example
    query = "give me the details on 3 feb 2023"  # Example query
    normalized_query = normalize_date_query(query)

    # Check if the date exists in the data
    if normalized_query in df['Date'].values:
        row_data = df[df['Date'] == normalized_query].iloc[0]
        answer = f"Date: {row_data['Date']}, Open: {row_data['Open']}, High: {row_data['High']}, Low: {row_data['Low']}, Close: {row_data['Close']}, Volume: {row_data['Volume']}"
    else:
        response = qa_chain.invoke({"query": query})
        answer = response["result"]  # Fallback to vector store if exact date not found

    print("Question:", query)
    print("Answer:", answer)


Question: give me the details on 3 feb 2023
Answer: Date: 03-02-2023, Open: 146.4895254643634, High: 155.74223078414798, Low: 146.29160978319123, Close: 152.89219665527344, Volume: 154357300
